<a href="https://colab.research.google.com/github/hyunshoon/bitcoin_price_prediction_with_sentiment/blob/main/Reddit_comment_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal
- Reddit의 Bticoin Subreddit, CryptoCurrency subreddit 대상
- 각 subreddit의 Daily Discussion URL list 생성
- Daily Discussion URL list를 통해 comment text crawling 가능


#Setting

In [9]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bioni

In [10]:
from selenium import webdriver
import time, re
from bs4 import BeautifulSoup

In [11]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

##########################################여기까지는 공통

#Crawling Sample

In [4]:
#해당 url로 이동
url = 'https://www.reddit.com/r/Bitcoin/search/?q=daily%20discussion&restrict_sr=1&sr_nsfw='#daily discussion으로 검색
url = 'https://www.reddit.com/r/cryptocurrency/search/?q=daily%20discussion&restrict_sr=1&sr_nsfw='#crypto currency 서브레딧
driver.get(url)

In [5]:
#페이지 끝까지 스크롤
time.sleep(1)
SCROLL_PAUSE_SEC = 2 #2초 대기 후 스크롤

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

KeyboardInterrupt: ignored

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [ ]:
soup.find_all(style =["color: rgb(135, 138, 140); cursor: pointer; fill: rgb(135, 138, 140);"])

In [ ]:
length = len(soup.find_all(style =["color: rgb(135, 138, 140); cursor: pointer; fill: rgb(135, 138, 140);"])) #

href_list = []
for i in range(length):
#  href_list.append(soup.find_all(style =['color:#878A8C;cursor:pointer;fill:#878A8C',"color: rgb(135, 138, 140); cursor: pointer; fill: rgb(135, 138, 140);"])[i].find("a")['href']) #original
  href_list.append(soup.find_all(style =["color: rgb(135, 138, 140); cursor: pointer; fill: rgb(135, 138, 140);"])[i].find_all(href=re.compile("https://www.reddit.com/r/CryptoCurrency/comments")))#cryptocurrency
  # href_list.append(soup.find_all(style =["color: rgb(135, 138, 140); cursor: pointer; fill: rgb(135, 138, 140);"])[i].find_all(href=re.compile("https://www.reddit.com/r/Bitcoin/comments")))#

In [ ]:
print(href_list[0])

In [ ]:
clean_href = []
for href in href_list:
  href_str = str(href)
  start = href_str.find('href')
  end = href_str.find('rel')
  clean_href.append(href_str[start+6:end-3])
clean_href

In [ ]:
#특정 월만 필터링 할 때
year = '2021'
month = 'october'
filterd_herf_list = [href for href in clean_href if href.find(month)!= -1 and href.find(year)!=1]
filterd_herf_list

# Bitcoin subreddit Crawling

############################스크롤 최대로 내려도 250개까지만 보여주는 문제 해결###################################
Cryptocurrency subreddit은 밑의 방식 사용했을 때 결측치가 많이 생김

In [23]:
#태그로 찾는 url_list
def cleansing(soup, month, year):
  length = len(soup.find_all('a'))
  total_list = []
  href_list = []
  for i in range(length):
    total_list.append(soup.find_all('a')[i]['href'])
  for li in total_list:
    if li.find("https://www.reddit.com/r/Bitcoin/comments")!= -1:
      href_list.append(li)

  clean_href = []

  for href in href_list:
    href_str = str(href)
    if (month and year) not in href_str:continue# 해당 월, 년 아니면 스킵
    start = href_str.find('href')
    end = href_str.find('rel')
    clean_href.append(href_str[start+1:end])
  #daily discussion, 해당 월, 년 포함되지 않은 url 필터링
  filterd_herf_list = [href for href in clean_href if href.find(month)!= -1 and href.find(year)!= -1 
                       and href.find('daily_discussion')!= -1]
  return  filterd_herf_list

In [26]:
month_list = ['january','february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
year_list = ['2017', '2018', '2019', '2020', '2021', '2022']
month_list = ['september', 'october', 'november', 'december']
year_list = ['2021']
total_url_list = []
for year in year_list:
  for month in month_list:
    url = 'https://www.reddit.com/r/Bitcoin/search/?q=daily%20discussion%2C%20'+month+'%2C%20'+year+'&restrict_sr=1&sr_nsfw='#bitcoin subreddit
    driver.get(url)

    #페이지 끝까지 스크롤
    time.sleep(1)
    SCROLL_PAUSE_SEC = 1 #1초 대기 후 스크롤

    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
      
    # total_url_list.append(cleansing(soup, month, year))
    total_url_list.append(cleansing(soup, month, year))

In [27]:
total_url_list

[['https://www.reddit.com/r/Bitcoin/comments/pjgc8e/daily_discussion_september_07_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pro3dl/daily_discussion_september_20_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pucy00/daily_discussion_september_24_2021',
  'https://www.reddit.com/r/Bitcoin/comments/psbyo9/daily_discussion_september_21_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pwz3vm/daily_discussion_september_28_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pxofhq/daily_discussion_september_29_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pycwin/daily_discussion_september_30_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pvn9rr/daily_discussion_september_26_2021',
  'https://www.reddit.com/r/Bitcoin/comments/ppttbw/daily_discussion_september_17_2021',
  'https://www.reddit.com/r/Bitcoin/comments/plehqb/daily_discussion_september_10_2021',
  'https://www.reddit.com/r/Bitcoin/comments/pk432t/daily_discussion_september_08_2021',
  'https://www.reddit

#Cryptocurrency subreddit Crawling

In [ ]:
#태그로 찾는 url_list
def cleansing(soup, month, year):
  length = len(soup.find_all('a'))
  total_list = []
  href_list = []
  for i in range(length):
    total_list.append(soup.find_all('a')[i]['href'])
  for li in total_list:
    if li.find("https://www.reddit.com/r/CryptoCurrency/comments")!= -1:
      href_list.append(li)

  clean_href = []

  for href in href_list:
    href_str = str(href)
    if (month and year) not in href_str:continue# 해당 월, 년 아니면 스킵
    start = href_str.find('href')
    end = href_str.find('rel')
    clean_href.append(href_str[start+1:end])
  #daily discussion, 해당 월, 년 포함되지 않은 url 필터링
  filterd_herf_list = [href for href in clean_href if href.find(month)!= -1 and href.find(year)!= -1 
                       and href.find('daily_discussion')!= -1]
  return  filterd_herf_list

In [ ]:
#daily discussion, month로 검색했을때 https://www.reddit.com/r/Bitcoin/search/?q=daily%20discussion%2C%20month&restrict_sr=1&sr_nsfw=
#daily discussion, february검색했을때 https://www.reddit.com/r/Bitcoin/search/?q=daily%20discussion%2C%20february&restrict_sr=1&sr_nsfw=
month_list = ['january','february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
year_list = ['2017', '2018', '2019', '2020', '2021', '2022']
year_list = ['2021', '2022']
total_url_list = []
for year in year_list:
  for month in month_list:
    url = 'https://www.reddit.com/r/CryptoCurrency/search/?q=daily%20discussion%20-%20'+month+'%20'+year+'&restrict_sr=1&sr_nsfw='#crypto subreddit
    
    driver.get(url)

    #페이지 끝까지 스크롤
    time.sleep(1)
    SCROLL_PAUSE_SEC = 1 #1초 대기 후 스크롤

    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
      
    # total_url_list.append(cleansing(soup, month, year))
    total_url_list.append(cleansing(soup, month, year))

In [ ]:
total_url_list

[[],
 ['https://www.reddit.com/r/CryptoCurrency/comments/5uu32x/daily_market_discussion_saturday_february_18_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/5uoqd9/daily_market_discussion_friday_february_17_2017'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['https://www.reddit.com/r/CryptoCurrency/comments/7112s7/daily_general_discussion_september_19_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/73de1q/daily_general_discussion_september_30_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/72amii/daily_general_discussion_september_25_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/7196hm/daily_general_discussion_september_20_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/71h8tw/daily_general_discussion_september_21_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/70tyv6/daily_general_discussion_september_18_2017',
  'https://www.reddit.com/r/CryptoCurrency/comments/7360os/daily_general_discussion_september_29_2017',
  'https://www.r

In [ ]:
len(total_url_list)

72